#Lab1
##0-1Knapsack

In [10]:
import numpy as np
from icecream import ic
from tqdm.notebook import tqdm
ic.configureOutput(includeContext=False, outputFunction=tqdm.write)

In [11]:
NUM_KNAPSACKS = 3
NUM_ITEMS = 10
NUM_DIMENSIONS = 2

In [12]:
VALUES = np.random.randint(0, 100, size=NUM_ITEMS)
WEIGHTS = np.random.randint(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = np.random.randint(0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS,NUM_DIMENSIONS))

In [13]:
print(VALUES)
print(WEIGHTS)
CONSTRAINTS

[77 53 39 60 62 16 47 29 34 85]
[[80 84]
 [17 24]
 [95 16]
 [32 28]
 [88 81]
 [29 88]
 [25  2]
 [37 36]
 [ 6 18]
 [68 72]]


array([[221, 331],
       [320, 141],
       [ 34, 190]])

###TEST PROBLEMS

In [14]:
#test some solutions
#start from classic knapsack solution
#random solution
def random_feasible_init(values, weights, constraints, num_knapsacks, rng=None):
    rng = np.random.default_rng() if rng is None else rng
    N, D = weights.shape; K = num_knapsacks
    sol = np.zeros((N, K), dtype=int)
    loads = np.zeros((K, D), dtype=int)

    # from low to high value/weight ratio
    norm = np.maximum(constraints.astype(float), 1.0)
    score = values / ((weights / norm).sum(axis=1) + 1e-9)
    order = np.argsort(score)                   

    for i in order:
        k = rng.integers(K)                     
        if np.all(loads[k] + weights[i] <= constraints):
            sol[i, k] = 1
            loads[k] += weights[i]
    return sol



In [15]:
def fitness(sol, values):
    # only return the fitness value
    return int((sol * values[:, None]).sum())

def knapsack_loads(sol, weights):
    return sol.T @ weights  # (K, D)

In [16]:
#local search
#neighbor
def best_improving_neighbor(sol, values, weights, constraints):
    import numpy as np
    N, K = sol.shape
    cur_fit = fitness(sol, values)
    loads = knapsack_loads(sol, weights)

    # 1. add i to some bag k
    for i in range(N):
        if sol[i].any():
            continue
        wi = weights[i]
        for k in range(K):
            if np.all(loads[k] + wi <= constraints):
                new_sol = sol.copy()
                new_sol[i, :] = 0
                new_sol[i, k] = 1
                gain = int(fitness(new_sol, values) - cur_fit)
                if gain > 0:
                    return new_sol, gain, {"op": "add", "i": i, "j": None, "k_from": None, "k_to": k}

    # 2. swap： swap i to bag k's j
    for k in range(K):
        for j in range(N):
            if sol[j, k] != 1:
                continue
            for i in range(N):
                if sol[i].any():
                    continue
                if np.all(loads[k] - weights[j] + weights[i] <= constraints):
                    new_sol = sol.copy()
                    new_sol[j, :] = 0
                    new_sol[i, :] = 0
                    new_sol[i, k] = 1
                    gain = int(fitness(new_sol, values) - cur_fit)
                    if gain > 0:
                        return new_sol, gain, {"op": "swap", "i": i, "j": j, "k_from": k, "k_to": k}

    return None, 0, None


In [17]:

ic.configureOutput(includeContext=False, outputFunction=tqdm.write)

def hill_climb(values, weights, constraints, num_knapsacks, *,
               restarts=20, max_iters=500, seed=42,
               debug=False, log_every=500,
               refresh_every=200):   
    rng = np.random.default_rng(seed)
    best_sol, best_fit = None, -1

    # total iterations for the progress bar
    TOTAL = restarts * max_iters
    bar = tqdm(total=TOTAL, desc="search", unit="try",
               dynamic_ncols=True, leave=True, position=0)
    bar.refresh()  # enforce refresh

    tries_done = 0  # total tries done so far

    for r in range(restarts):
        sol = random_feasible_init(values, weights, constraints, num_knapsacks, rng)
        cur_fit = fitness(sol, values)

        it = 0
        while it < max_iters:
            tries_done += 1
            bar.update(1)

            if tries_done % refresh_every == 0:
                bar.set_postfix_str(f"r={r+1} fit={int(cur_fit)}")
                bar.refresh()  

            neighbor, gain, info = best_improving_neighbor(sol, values, weights, constraints)
            if neighbor is None or gain <= 0:
                break  

            # accept the neighbor
            sol = neighbor
            cur_fit = fitness(sol, values)
            it += 1

            if debug and (it % log_every == 0):
                ic(f"r{r+1} it={it} {info.get('op')} i={info.get('i')} "
                   f"j={info.get('j')} k={info.get('k_to')} "
                   f"gain=+{int(gain)} fit={int(cur_fit)}")

        if cur_fit > best_fit:
            best_fit, best_sol = cur_fit, sol

    #end bar
    bar.total = bar.n
    bar.close()
    loads = knapsack_loads(best_sol, weights)  # (K, D)
    assigned = np.full(best_sol.shape[0], -1, dtype=int)
    mask = best_sol.any(axis=1)
    assigned[mask] = best_sol.argmax(axis=1)[mask]

    lines = []
    lines.append(f"BEST fit: {int(best_fit)}")
    lines.append(f"cap: {constraints.tolist()}")
    lines.append("loads:")
    for k in range(best_sol.shape[1]):
        lines.append(f"  k{k}: {loads[k].tolist()}")

    lines.append("bags:")
    for k in range(best_sol.shape[1]):
        items_k = np.where(assigned == k)[0]
        val_k = int(values[items_k].sum())
        lines.append(f"  k{k}: items={items_k.tolist()}  value={val_k}")

    ic("\n".join(lines))

    return best_sol, int(best_fit)

    


In [18]:
# Problem 1:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=NUM_DIMENSIONS)



sol = random_feasible_init(VALUES, WEIGHTS, CONSTRAINTS, NUM_KNAPSACKS, rng)
print("Initial solution:")
print(sol)
sol, fit = hill_climb(
    VALUES, WEIGHTS, CONSTRAINTS, NUM_KNAPSACKS,
    restarts=20, max_iters=5000, seed=42,
    debug=False    
)






Initial solution:
[[0 1 0]
 [1 0 0]
 [0 0 1]
 [1 0 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [1 0 0]
 [1 0 0]
 [0 0 1]
 [0 1 0]
 [0 0 1]
 [0 1 0]
 [1 0 0]
 [1 0 0]
 [0 1 0]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 1 0]]


search:   0%|          | 0/100000 [00:00<?, ?try/s]

ic| "\n".join(lines): '''BEST fit: 1065
                       cap: [614, 496]
                       loads:
                         k0: [323, 387]
                         k1: [279, 257]
                         k2: [383, 428]
                       bags:
                         k0: items=[0, 1, 2, 3, 11, 14]  value=361
                         k1: items=[5, 6, 8, 10, 15, 17]  value=255
                         k2: items=[4, 7, 9, 12, 13, 16, 18, 19]  value=449'''


In [19]:
# Problem 2:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(1000 * 2, 1000 * NUM_ITEMS // NUM_KNAPSACKS, size=NUM_DIMENSIONS)




sol = random_feasible_init(VALUES, WEIGHTS, CONSTRAINTS, NUM_KNAPSACKS, rng)
print("Initial solution:")
print(sol)
sol, fit = hill_climb(
    VALUES, WEIGHTS, CONSTRAINTS, NUM_KNAPSACKS,
    restarts=20, max_iters=50, seed=42,
    debug=False     
)



Initial solution:
[[0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 

search:   0%|          | 0/1000 [00:00<?, ?try/s]

ic| "\n".join(lines): '''BEST fit: 35406
                       cap: [5837, 5754, 9351, 3205, 4450, 3447, 3952, 9256, 9525, 2357]
                       loads:
                         k0: [2770, 4715, 3941, 3172, 3961, 3020, 3525, 5187, 4185, 2269]
                         k1: [2296, 2889, 2785, 3182, 2846, 2425, 1535, 2771, 3723, 2318]
                         k2: [5066, 4299, 3094, 2436, 3967, 3126, 3722, 3218, 4533, 2345]
                         k3: [2345, 3774, 3540, 2986, 3458, 2984, 3663, 3464, 2509, 2302]
                         k4: [3380, 3039, 3563, 3135, 3670, 3322, 2810, 1808, 2884, 2251]
                         k5: [4320, 2845, 4447, 3151, 3804, 3313, 3565, 3288, 3092, 1861]
                         k6: [4902, 3360, 1879, 2263, 3695, 3107, 2288, 1410, 3093, 2327]
                         k7: [2648, 3746, 3721, 2914, 2690, 2910, 3343, 1686, 2517, 2344]
                         k8: [1663, 3189, 2205, 2772, 2100, 2617, 1957, 1917, 2158, 2225]
                         k9: [

In [20]:
# Problem 3:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(1000 * 10, 1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS, size=NUM_DIMENSIONS)



sol = random_feasible_init(VALUES, WEIGHTS, CONSTRAINTS, NUM_KNAPSACKS, rng)
print("Initial solution:")
print(sol)
sol, fit = hill_climb(
    VALUES, WEIGHTS, CONSTRAINTS, NUM_KNAPSACKS,
    restarts=20, max_iters=50, seed=42,
    debug=True 
)



Initial solution:
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


search:   0%|          | 0/1000 [00:00<?, ?try/s]

ic| "\n".join(lines): '''BEST fit: 619154
                       cap: [79834, 52005, 17527, 30007, 59295, 98921, 12881, 36174, 63958, 90566, 25740, 12251, 27746, 78136, 90704, 39249, 23072, 69010, 17326, 29277, 42886, 48016, 74931, 93479, 30898, 18137, 75557, 56030, 75506, 90246, 42093, 34668, 22075, 79709, 98971, 15032, 57157, 21369, 76865, 11589, 82666, 78074, 68715, 85558, 61242, 80943, 26905, 84378, 85069, 77221, 16434, 28156, 17827, 83548, 72921, 27512, 41395, 23936, 72519, 96465, 12904, 72709, 81023, 81015, 28959, 98041, 90482, 89802, 76408, 29899, 51171, 94767, 78351, 57860, 58504, 40545, 97025, 99185, 42888, 54965, 24945, 51281, 31276, 61913, 96714, 93034, 72901, 95222, 37857, 53461, 15463, 71355, 64979, 70086, 48286, 38026, 60109, 78500, 56643, 69915]
                       loads:
                         k0: [12217, 14126, 12675, 11430, 12236, 11781, 12792, 13168, 14663, 13071, 13431, 11871, 10322, 12419, 10333, 10963, 8331, 12444, 12518, 13970, 10955, 14566, 12591, 12928, 12

In [ ]:
#testing output

print(VALUES)
print(WEIGHTS)
print(CONSTRAINTS)

[ 89 773 654 ... 819 678 498]
[[197  33 627 ... 319 937 293]
 [550 543 242 ... 942 568 950]
 [436 411 836 ... 664 169 846]
 ...
 [869 582 968 ... 567 291 288]
 [ 97 220 662 ... 171 310 833]
 [656  51 146 ... 286 725 372]]
[79834 52005 17527 30007 59295 98921 12881 36174 63958 90566 25740 12251
 27746 78136 90704 39249 23072 69010 17326 29277 42886 48016 74931 93479
 30898 18137 75557 56030 75506 90246 42093 34668 22075 79709 98971 15032
 57157 21369 76865 11589 82666 78074 68715 85558 61242 80943 26905 84378
 85069 77221 16434 28156 17827 83548 72921 27512 41395 23936 72519 96465
 12904 72709 81023 81015 28959 98041 90482 89802 76408 29899 51171 94767
 78351 57860 58504 40545 97025 99185 42888 54965 24945 51281 31276 61913
 96714 93034 72901 95222 37857 53461 15463 71355 64979 70086 48286 38026
 60109 78500 56643 69915]
